In [107]:
import geopandas as gpd
shapefile = './data/countries_110m/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

country country_code  \
0                         Fiji          FJI   
1  United Republic of Tanzania          TZA   
2               Western Sahara          SAH   
3                       Canada          CAN   
4     United States of America          USA   

                                            geometry  
0  (POLYGON ((180 -16.06713266364245, 180 -16.555...  
1  POLYGON ((33.90371119710453 -0.950000000000000...  
2  POLYGON ((-8.665589565454809 27.65642588959236...  
3  (POLYGON ((-122.84 49.00000000000011, -122.974...  
4  (POLYGON ((-122.84 49.00000000000011, -120 49....

In [109]:
print(gdf[gdf['country'] == 'Antarctica'])
#Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])

Empty GeoDataFrame
Columns: [country, country_code, geometry]
Index: []


In [110]:
grid_crs=gdf.crs
grid_crs

{'init': 'epsg:4326'}

In [111]:
import json
#Read data to json.
gdf_json = json.loads(gdf.to_json())
#Convert to String like object.
grid = json.dumps(gdf_json)

In [112]:
import pandas as pd
datafile = './data/coronavirus.csv'
#Read csv file using pandas
df = pd.read_csv(datafile, names = ['continent', 'country/territory', 'confirmed_cases', 'deaths'], skiprows = 1)
df.head()

continent  country/territory  confirmed_cases  deaths
0      Asia              China            37242     812
1      Asia          Singapore               40       0
2      Asia           Thailand               32       0
3      Asia              Japan               26       0
4      Asia  Republic of Korea               25       0

In [113]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_Dashboard")
import numpy as np
def geolocate(city=None, country=None):
    '''
    Inputs city and country, or just country. Returns the lat/long coordinates of 
    either the city if possible, if not, then returns lat/long of the center of the country.
    '''
    try:
            # Geolocate the center of the country
        loc = geolocator.geocode(country)
            # And return latitude and longitude 
        return (loc.latitude, loc.longitude)
        # Otherwise
    except:
            # Return missing value
        return np.nan
        

In [115]:
geolocate(country='China')

(35.000074, 104.999927)

In [116]:
df['location'] = df['country/territory'].apply(geolocate)


In [117]:
df['scaled_confirmed_cases'] = np.log10(df['confirmed_cases'])*10
df

continent                           country/territory  confirmed_cases  \
0       Asia                                       China            37242   
1       Asia                                   Singapore               40   
2       Asia                                    Thailand               32   
3       Asia                                       Japan               26   
4       Asia                           Republic of Korea               25   
5       Asia                                      Taiwan               18   
6       Asia                                    Malaysia               16   
7       Asia                                     Vietnam               14   
8       Asia                        United Arab Emirates                7   
9       Asia                                       India                3   
10      Asia                                 Philippines                3   
11      Asia                                       Nepal                1   
12      Asia                                   Sri Lanka                1   
13      Asia                                    Cambodia                1   
14     Other  Cases on an international conveyance Japan               64   
15    Europe                                     Germany               14   
16    Europe                                      France               11   
17    Europe                                       Italy                3   
18    Europe                              United Kingdom                3   
19    Europe                                      Russia                2   
20    Europe                                     Finland                1   
21    Europe                                     Belgium                1   
22    Europe                                       Spain                1   
23    Europe                                      Sweden                1   
24   America                    United States of America               12   
25   America                                      Canada                7   
26   Oceania                                   Australia               15   

    deaths              locations               location  \
0      812  (44.933143, 7.540121)                    NaN   
1        0  (44.933143, 7.540121)                    NaN   
2        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
3        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
4        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
5        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
6        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
7        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
8        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
9        0  (44.933143, 7.540121)  (44.933143, 7.540121)   
10       1  (44.933143, 7.540121)                    NaN   
11       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
12       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
13       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
14       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
15       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
16       0  (44.933143, 7.540121)                    NaN   
17       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
18       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
19       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
20       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
21       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
22       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
23       0  (44.933143, 7.540121)                    NaN   
24       0  (44.933143, 7.540121)                    NaN   
25       0  (44.933143, 7.540121)  (44.933143, 7.540121)   
26       0  (44.933143, 7.540121)                    NaN   

    scaled_confirmed_cases  
0                45.710330  
1                16.020600  
2                15.051500  
3                14.149733  
4                13.

In [91]:
df[['lat','lon']] = pd.DataFrame(df['locations'].tolist(),index=df.index)
df.head()



continent  country/territory  confirmed_cases  deaths  \
0      Asia              China            37242     812   
1      Asia          Singapore               40       0   
2      Asia           Thailand               32       0   
3      Asia              Japan               26       0   
4      Asia  Republic of Korea               25       0   

               locations  scaled_confirmed_cases        lat       lon  
0  (44.933143, 7.540121)               45.710330  44.933143  7.540121  
1  (44.933143, 7.540121)               16.020600  44.933143  7.540121  
2  (44.933143, 7.540121)               15.051500  44.933143  7.540121  
3  (44.933143, 7.540121)               14.149733  44.933143  7.540121  
4  (44.933143, 7.540121)               13.979400  44.933143  7.540121

In [92]:
datapoints=df.drop(['locations'], axis=1)
datapoints.head()

continent  country/territory  confirmed_cases  deaths  \
0      Asia              China            37242     812   
1      Asia          Singapore               40       0   
2      Asia           Thailand               32       0   
3      Asia              Japan               26       0   
4      Asia  Republic of Korea               25       0   

   scaled_confirmed_cases        lat       lon  
0               45.710330  44.933143  7.540121  
1               16.020600  44.933143  7.540121  
2               15.051500  44.933143  7.540121  
3               14.149733  44.933143  7.540121  
4               13.979400  44.933143  7.540121

In [96]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, ColumnDataSource
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = grid)
pointsource = ColumnDataSource(datapoints)

#Create figure object.
p = figure(title = 'Geographic distribution of 2019-nCov cases worldwide', plot_height = 600 , plot_width = 950)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = 'lightgrey',
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

p.circle('lat','lon',source=pointsource, size=15)
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

Loading BokehJS ...